In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Oct  8 06:20:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1ccoCOvxD2hFm1ejOVRDlT2SivkoNtC6V

Downloading...
From: https://drive.google.com/uc?id=1ccoCOvxD2hFm1ejOVRDlT2SivkoNtC6V
To: /content/ml-100k.zip
100% 4.92M/4.92M [00:00<00:00, 66.4MB/s]


In [ ]:
!unzip "ml-100k.zip"  -d  "/content"

Archive:  ml-100k.zip
   creating: /content/ml-100k/
  inflating: /content/ml-100k/allbut.pl  
  inflating: /content/ml-100k/mku.sh  
  inflating: /content/ml-100k/README  
  inflating: /content/ml-100k/u.data  
  inflating: /content/ml-100k/u.genre  
  inflating: /content/ml-100k/u.info  
  inflating: /content/ml-100k/u.item  
  inflating: /content/ml-100k/u.occupation  
  inflating: /content/ml-100k/u.user  
  inflating: /content/ml-100k/u1.base  
  inflating: /content/ml-100k/u1.test  
  inflating: /content/ml-100k/u2.base  
  inflating: /content/ml-100k/u2.test  
  inflating: /content/ml-100k/u3.base  
  inflating: /content/ml-100k/u3.test  
  inflating: /content/ml-100k/u4.base  
  inflating: /content/ml-100k/u4.test  
  inflating: /content/ml-100k/u5.base  
  inflating: /content/ml-100k/u5.test  
  inflating: /content/ml-100k/ua.base  
  inflating: /content/ml-100k/ua.test  
  inflating: /content/ml-100k/ub.base  
  inflating: /content/ml-100k/ub.test  


In [ ]:
!mv ml-100k/* ./

In [ ]:
dateparse = lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

rating_df = pd.read_csv('u.data', sep='\t', 
                        names=['userID', 'itemID', 'rating', 'time'], 
                       )

movie_df = pd.read_csv('u.item', sep='|', encoding='latin-1',
                    names=['itemID', 'movie', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'action', 
                           'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 
                           'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'])

user_df = pd.read_csv('u.user', sep='|', encoding='latin-1',
                     names=['userID', 'age', 'gender', 'occupation', 'zip_code'])


In [ ]:
rating_df.dropna(inplace=True)

In [ ]:

rating_df["itemID"] = rating_df["itemID"].astype(str)
rating_df["userID"] = rating_df["userID"].astype(str)

### Prepare dataset

In [ ]:
unique_users = rating_df.userID.unique()
user_ids = dict(zip(unique_users, np.arange(unique_users.shape[0], dtype=np.int32)))

unique_items = rating_df.itemID.unique()
item_ids = dict(zip(unique_items, np.arange(unique_items.shape[0], dtype=np.int32)))

In [ ]:
product_ids = list(set(list(rating_df.itemID.unique())))
user_ids = list(set(list(rating_df.userID.unique())))

In [ ]:
x = rating_df.groupby('itemID').count()['rating'] > 50
itemIndex = x[x].index
nm_rating_df=rating_df[rating_df['itemID'].isin(itemIndex)]
y = nm_rating_df.groupby('userID').count()['rating']>=50
users = y[y].index
final_ratings = nm_rating_df[nm_rating_df['userID'].isin(users)]

In [ ]:
!pip install recommenders
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 11.6 MB/s 
     |████████████████████████████████| 636 kB 58.4 MB/s 
     |████████████████████████████████| 11.8 MB 34.8 MB/s 
     |████████████████████████████████| 4.9 MB 54.1 MB/s 
     |████████████████████████████████| 61 kB 7.6 MB/s 
     |████████████████████████████████| 12.4 MB 31.3 MB/s 
     |████████████████████████████████| 355 kB 65.9 MB/s 
     |████████████████████████████████| 120 kB 2.8 MB/s 
     |████████████████████████████████| 310 kB 66.5 MB/s 
     |████████████████████████████████| 395 kB 72.3 MB/s 
     |████████████████████████████████| 6.6 MB 65.0 MB/s 
     |████████████████████████████████| 163 kB 75.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705374 sha256=7d9e6df94e608d55d53250c1a689093cddba636fdcf2f96a4eaf1f46e0fc09b4
  Stored in directory: /root/.cache/pip/whe

In [ ]:

from collections import defaultdict
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet



In [ ]:
ncf_df=final_ratings[['userID','itemID','rating','time']]

In [ ]:
item_ids = list(set(list(ncf_df.itemID.unique())))
user_ids = list(set(list(ncf_df.userID.unique())))

In [ ]:
dict_users = {}
index = 0
for ids in sorted(user_ids):
    dict_users[ids] = index
    index += 1
dict_items = {}
index = 0
for ids in sorted(item_ids):
    dict_items[ids] = index
    index += 1

In [ ]:
ncf_df['userID'] = ncf_df.userID.map(dict_users)
ncf_df['itemID'] = ncf_df.itemID.map(dict_items)

In [ ]:
final_ratings_NCF=ncf_df.sort_values(by=['userID','time'], ascending=[False,False]) 
final_ratings_NCF.shape

(71065, 4)

In [ ]:

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 30     # number of negative examples per positive example
model_name = 'ssept'  # 'sasrec' or 'ssept'

In [ ]:
final_ratings_NCF=final_ratings_NCF[['userID','itemID','time']]

In [ ]:

outfile = "input_sasrec" + '.txt'


In [ ]:
final_ratings_NCF = final_ratings_NCF.sort_values(by=["userID", "time"])
final_ratings_NCF.drop(columns=["time"], inplace=True)
final_ratings_NCF.to_csv( outfile, sep="\t", header=False, index=False)

### SASREC

In [ ]:
data = SASRecDataSet(filename=outfile, col_sep="\t")

In [ ]:
num_epochs = 3
batch_size = 512

In [ ]:
# create train, validation and test splits
data.split()

# some statistics
num_steps = int(len(data.user_train) / batch_size)
cc = 0.0
for u in data.user_train:
    cc += len(data.user_train[u])
print('%g Users and %g items' % (data.usernum, data.itemnum))
print('average sequence length: %.2f' % (cc / len(data.user_train)))

523 Users and 592 items
average sequence length: 133.62


In [ ]:
model = SASREC(item_num=data.itemnum,
                   seq_max_len=maxlen,
                   num_blocks=num_blocks,
                   embedding_dim=hidden_units,
                   attention_dim=hidden_units,
                   attention_num_heads=num_heads,
                   dropout_rate=dropout_rate,
                   conv_dims = [100, 100],
                   l2_reg=l2_emb,
                   num_neg_test=num_neg_test
    )

In [ ]:
#the sampler generates a sequence of negative items of the same length as the original history

sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)


In [ ]:
with Timer() as train_time:
    t_test = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))


epoch: 3, test (NDCG@10: 0.19058153586268312, HR@10: 0.3824091778202677)
Time cost for training is 0.66 mins


### SSEPT

In [ ]:
model = SSEPT(item_num=data.itemnum,
                  user_num=data.usernum,
                  seq_max_len=maxlen,
                  num_blocks=num_blocks,
                  # embedding_dim=hidden_units,  # optional
                  user_embedding_dim=10,
                  item_embedding_dim=hidden_units,
                  attention_dim=hidden_units,
                  attention_num_heads=num_heads,
                  dropout_rate=dropout_rate,
                  conv_dims = [110, 110],
                  l2_reg=l2_emb,
                  num_neg_test=num_neg_test
    )

In [ ]:
with Timer() as train_time:
    t_test = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))


epoch: 3, test (NDCG@10: 0.18314568304031467, HR@10: 0.3824091778202677)
Time cost for training is 0.36 mins
